# DeepFM

이번 실습에서는 DeepFM 모델을 이해하고 구현해보겠습니다.  

DeepFM 모델은 Factorization machines와 neural network를 합친 모델로, Wide & Deep model과 유사하지만, feature engineering이 필요하지 않다는 특징을 가지고 있습니다.  
<br/>
사용자가 영화에 대해 Rating한 데이터, 영화의 장르 데이터를 이용하여 Train/Test data를 생성한 다음, Train data로 학습한 모델을 Test data에 대해 평가해봅니다.   
사용한 데이터는 Implicit feedback data로, 사용자가 시청한 영화(Positive instances)는 rating = 1로 기록됩니다. 따라서 시청하지 않은 영화에 대해 각 유저별로 Negative instances sampling을 진행합니다.   
<br/>
**구현에 앞서, DeepFM 논문을 꼭 읽어보시길 권장합니다.**

* 참고  
    - DeepFM: A Factorization-Machine based Neural Network for CTR Prediction (https://arxiv.org/pdf/1703.04247.pdf)  
    - Wide & Deep Learning for Recommender Systems (https://arxiv.org/pdf/1606.07792.pdf)
    - Factorization Machines (https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5694074)
    - https://d2l.ai/chapter_recommender-systems/deepfm.html

# Modules

In [1]:
import csv
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Data preprocessing
0. Dataset 다운로드  
<br/>
1. Rating df 생성  
rating 데이터(train_ratings.csv)를 불러와 [user, item, rating]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
2. Genre df 생성   
genre 정보가 담긴 데이터(genres.tsv)를 불러와 genre이름을 id로 변경하고, [item, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.    
<br/>
3. Negative instances 생성   
rating 데이터는 implicit feedback data(rating :0/1)로, positive instances로 구성되어 있습니다. 따라서 rating이 없는 item중 negative instances를 뽑아서 데이터에 추가하게 됩니다.   
<br/>
4. Join dfs   
rating df와 genre df를 join하여 [user, item, rating, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
5. zero-based index로 mapping   
Embedding을 위해서 user,item,genre를 zero-based index로 mapping합니다.
    - user : 0-31359
    - item : 0-6806
    - genre : 0-17  
<br/>
6. feature matrix X, label tensor y 생성   
[user, item, genre] 3개의 field로 구성된 feature matrix를 생성합니다.   
<br/>
7. data loader 생성

## 데이터 다운로드
이곳에 대회 사이트(AI Stages)에 있는 data의 URL을 입력해주세요. 
- 데이터 URL은 변경될 수 있습니다.
- 예) `!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000176/data/data.tar.gz`

In [2]:
# 0. Dataset 다운로드
!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000179/data/data.tar.gz
!tar -xf data.tar.gz

--2022-04-05 17:41:49--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000179/data/data.tar.gz
Resolving aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)... 52.92.194.65
Connecting to aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)|52.92.194.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33425907 (32M) [binary/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]  31.88M  11.5MB/s    in 2.8s    

2022-04-05 17:41:52 (11.5 MB/s) - ‘data.tar.gz’ saved [33425907/33425907]

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ig

In [2]:
# 1. Rating df 생성
rating_data = "/opt/ml/train_pos_ratings.csv"

raw_rating_df = pd.read_csv(rating_data)
raw_rating_df
raw_rating_df['rating'] = 1.0 # implicit feedback

raw_neg_rating_data = "/opt/ml/train_neg_ratings.csv"
raw_neg_rating_df = pd.read_csv(raw_neg_rating_data)
raw_neg_rating_df['rating'] = 0.0 # implicit feedback

users = set(raw_rating_df.loc[:, 'user'])
items = set(raw_rating_df.loc[:, 'item'])
# times = list(set(raw_rating_df.loc[:, 'time']))

# #2. Time df 생성
# raw_time_df = raw_rating_df[["user","item","time"]]
# raw_rating_df.drop(['time'],axis=1,inplace=True)

#2. Genre df 생성
genre_data = "/opt/ml/input/data/train/genres.tsv"

raw_genre_df = pd.read_csv(genre_data, sep='\t')
raw_genre_df = raw_genre_df.drop_duplicates(subset=['item']) #item별 하나의 장르만 남도록 drop
genre_dict = {genre:i for i, genre in enumerate(set(raw_genre_df['genre']))}
raw_genre_df['genre']  = raw_genre_df['genre'].map(lambda x : genre_dict[x]) #genre id로 변경

#2. year df 생성
year_data = "/opt/ml/input/data/train/years.tsv"

raw_year_df = pd.read_csv(year_data, sep='\t')
year_dict = {year:i for i, year in enumerate(set(raw_year_df['year']))}
raw_year_df['year']  = raw_year_df['year'].map(lambda x : year_dict[x]) #year id로 변경

#2. director df 생성
director_data = "/opt/ml/input/data/train/directors.tsv"

raw_director_df = pd.read_csv(director_data, sep='\t')
raw_director_df = raw_director_df.drop_duplicates(subset=['item']) #item별 하나의 감독만 남도록 drop
director_dict = {director:i for i, director in enumerate(set(raw_director_df['director']))}
raw_director_df['director']  = raw_director_df['director'].map(lambda x : director_dict[x]) #director id로 변경


#2. writer df 생성
writer_data = "/opt/ml/input/data/train/writers.tsv"

raw_writer_df = pd.read_csv(writer_data, sep='\t')
raw_writer_df = raw_writer_df.drop_duplicates(subset=['item']) #item별 하나의 각본가만 남도록 drop
writer_dict = {writer:i for i, writer in enumerate(set(raw_writer_df['writer']))}
raw_writer_df['writer']  = raw_writer_df['writer'].map(lambda x : writer_dict[x]) #writer id로 변경

### 3. Negative instance 생성 & join(Pass 하기)

In [3]:

print("Create Nagetive instances")
# num_negative = 50
user_group_dfs = list(raw_rating_df.groupby('user')['item'])
user_group_dfs

Create Nagetive instances


[(11,
  0      4370
  1     40815
  2         2
  3      7373
  4      2617
  5      4886
  6      2174
  7      1127
  8     33004
  9     31696
  10     7438
  11    34405
  12     6754
  13     2987
  14     6373
  15     2167
  16     8961
  17     4874
  18       34
  19     2115
  20     5971
  21     8360
  22     6350
  23    46530
  24     4270
  25    37729
  26     5254
  27      733
  28       47
  29     1485
  30    32587
  31     8861
  32    37386
  33    60074
  34     5218
  35      590
  36     2288
  37     3703
  38     1377
  39     1215
  40      292
  41     2054
  42     3996
  43     1206
  44      924
  45     2021
  46     1923
  47     3081
  48     7454
  49     7147
  Name: item, dtype: int64),
 (14,
  50      588
  51     1198
  52     1035
  53     1028
  54      914
  55     1223
  56     2011
  57      919
  58     1907
  59     2081
  60     1073
  61       34
  62     2080
  63     4016
  64     8368
  65     5816
  66     1302
  67      539
  68   

In [5]:

first_row = True
user_neg_dfs = pd.DataFrame()

for u, u_items in tqdm(user_group_dfs):
    u_items = set(u_items)
    num_negative = len(u_items)
    i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    
    i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)
        
# user_neg_dfs['time'] = raw_rating_df['time']
joined_rating_df = pd.concat([raw_rating_df, user_neg_dfs], axis = 0, sort=False)
joined_rating_df
# joined_rating_df = pd.merge(joined_rating_df, raw_genre_df, left_on='item', right_on='item', how='inner')
# joined_rating_df = pd.merge(joined_rating_df, raw_year_df, left_on='item', right_on='item', how='inner')
# joined_rating_df = pd.merge(joined_rating_df, raw_director_df, left_on='item', right_on='item', how='inner')
# joined_rating_df = pd.merge(joined_rating_df, raw_writer_df, left_on='item', right_on='item', how='inner')
# 4. Join dfs
# print("Joined rating df")
# print(joined_rating_df)

100%|██████████| 31360/31360 [11:07<00:00, 46.99it/s]


,user,item,rating
0,11,4370,1.0
1,11,40815,1.0
2,11,2,1.0
3,11,7373,1.0
4,11,2617,1.0
...,...,...,...
95,138493,6666,0.0
96,138493,8641,0.0
97,138493,1635,0.0
98,138493,64716,0.0


In [7]:
joined_rating_df.to_csv("/opt/ml/ratings.csv")

### join

In [3]:
raw_neg_rating_df

,user,item,rating
0,11,42738,0.0
1,11,54272,0.0
2,11,54259,0.0
3,11,48780,0.0
4,11,59615,0.0
...,...,...,...
1567995,138493,4085,0.0
1567996,138493,377,0.0
1567997,138493,8464,0.0
1567998,138493,33004,0.0


In [4]:
joined_rating_df = pd.concat([raw_rating_df, raw_neg_rating_df], axis = 0, sort=False)
joined_rating_df = pd.merge(joined_rating_df, raw_genre_df, left_on='item', right_on='item', how='inner')
joined_rating_df = pd.merge(joined_rating_df, raw_year_df, left_on='item', right_on='item', how='inner')
joined_rating_df = pd.merge(joined_rating_df, raw_director_df, left_on='item', right_on='item', how='inner')
joined_rating_df = pd.merge(joined_rating_df, raw_writer_df, left_on='item', right_on='item', how='inner')

In [5]:
joined_rating_df

,user,item,rating,genre,year,director,writer
0,11,4370,1.0,5,79,1251,1978
1,189,4370,1.0,5,79,1251,1978
2,318,4370,1.0,5,79,1251,1978
3,395,4370,1.0,5,79,1251,1978
4,430,4370,1.0,5,79,1251,1978
...,...,...,...,...,...,...,...
3000439,131894,2550,0.0,1,41,928,1786
3000440,131894,849,0.0,12,74,425,704
3000441,131894,74685,0.0,12,88,62,567
3000442,131894,3726,0.0,12,54,425,704


### Zero-indexing

In [6]:

# 5. user, item을 zero-based index로 mapping
users = list(set(joined_rating_df.loc[:,'user']))
users.sort()
items =  list(set((joined_rating_df.loc[:, 'item'])))
items.sort()
genres =  list(set((joined_rating_df.loc[:, 'genre'])))
genres.sort()
years =  list(set((joined_rating_df.loc[:, 'year'])))
years.sort()
directors =  list(set((joined_rating_df.loc[:, 'director'])))
directors.sort()
writers =  list(set((joined_rating_df.loc[:, 'writer'])))
writers.sort()

if len(users)-1 != max(users):
    users_dict = {users[i]: i for i in range(len(users))}
    joined_rating_df['user']  = joined_rating_df['user'].map(lambda x : users_dict[x])
    users = list(set(joined_rating_df.loc[:,'user']))
    
if len(items)-1 != max(items):
    items_dict = {items[i]: i for i in range(len(items))}
    joined_rating_df['item']  = joined_rating_df['item'].map(lambda x : items_dict[x])
    items =  list(set((joined_rating_df.loc[:, 'item'])))

if len(genres)-1 != max(genres):
    genres_dict = {genres[i]: i for i in range(len(genres))}
    joined_rating_df['genre']  = joined_rating_df['genre'].map(lambda x : genres_dict[x])
    genres =  list(set((joined_rating_df.loc[:, 'genre'])))

if len(years)-1 != max(years):
    years_dict = {years[i]: i for i in range(len(years))}
    joined_rating_df['year']  = joined_rating_df['year'].map(lambda x : years_dict[x])
    years =  list(set((joined_rating_df.loc[:, 'year'])))

if len(directors)-1 != max(directors):
    directors_dict = {directors[i]: i for i in range(len(directors))}
    joined_rating_df['director']  = joined_rating_df['director'].map(lambda x : directors_dict[x])
    directors =  list(set((joined_rating_df.loc[:, 'director'])))
    
if len(writers)-1 != max(writers):
    writers_dict = {writers[i]: i for i in range(len(writers))}
    joined_rating_df['writer']  = joined_rating_df['writer'].map(lambda x : writers_dict[x])
    writers =  list(set((joined_rating_df.loc[:, 'writer'])))
    
joined_rating_df = joined_rating_df.sort_values(by=['user'])
joined_rating_df.reset_index(drop=True, inplace=True)

data = joined_rating_df
print("Data")
print(data)

n_data = len(data)
n_user = len(users)
n_item = len(items)
n_genre = len(genres)
n_year = len(years)
n_director = len(directors)
n_writers = len(writers)

# print("# of data : {}\n# of users : {}\n# of items : {}\n# of genres : {}".format(n_data, n_user, n_item, n_genre))

Data
          user  item  rating  genre  year  director  writer
0            0   892     1.0      6    78       502     656
1            0  1269     0.0      9    80       585     758
2            0  1386     1.0      1    83       595     768
3            0   687     1.0      8    76       230     778
4            0   775     0.0      1    64       216      58
...        ...   ...     ...    ...   ...       ...     ...
2924068  31359  1378     1.0      1    83       658     775
2924069  31359  1146     0.0      1    81       488     675
2924070  31359  1252     0.0      2    24       204     259
2924071  31359  1675     0.0      1    88       360     572
2924072  31359   913     1.0      2    78       156     202

[2924073 rows x 7 columns]


In [21]:
#6. feature matrix X, label tensor y 생성
user_col = torch.tensor(data.loc[:,'user'])
item_col = torch.tensor(data.loc[:,'item'])
genre_col = torch.tensor(data.loc[:,'genre'])
year_col = torch.tensor(data.loc[:,'year'])
director_col = torch.tensor(data.loc[:,'director'])
writer_col = torch.tensor(data.loc[:,'writer'])

offsets = [0, n_user, n_user+n_item, n_user+n_item+n_genre, n_user+n_item+n_genre+n_year, n_user+n_item+n_genre+n_year+n_director]
for col, offset in zip([user_col, item_col, genre_col, year_col, director_col, writer_col], offsets):
    col += offset

# X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1), year_col.unsqueeze(1), director_col.unsqueeze(1), writer_col.unsqueeze(1)], dim=1)
X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1), year_col.unsqueeze(1)], dim=1)
y = torch.tensor(list(data.loc[:,'rating']))


#7. data loader 생성
class RatingDataset(Dataset):
    def __init__(self, input_tensor, target_tensor):
        self.input_tensor = input_tensor.long()
        self.target_tensor = target_tensor.long()

    def __getitem__(self, index):
        return self.input_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.target_tensor.size(0)


dataset = RatingDataset(X, y)
train_ratio = 0.9

train_size = int(train_ratio * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [22]:
dataset[0]

(tensor([    0, 32252, 33104, 33193]), tensor(1))

   # Model architecture (DeepFM)
   DeepFM 모델은 1) FM component와  2) Deep component가 병렬적으로 결합되어 있습니다. 구조는 다음과 같습니다.
<img src='https://drive.google.com/uc?id=1vwcxUJQTIsg5QH9CuH5PcUEfExhToUHR'>  
각 구조는 다음과 같습니다.  
   **1. FM component**  
       FM component는 우리가 아는 2-way Factorization machines(degree=2)입니다. FM은 variables 간의 interaction을 다음과 같이 모델링 합니다.   
     **<center> equation (1) </center>**
   $$\hat{y}(x):=w_0 + \sum_{i=1}^{n}w_ix_i + \sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j$$   
   이때, 세번째 interaction term을 전개하여 다음과 같이 쓸 수 있습니다.(논문 참고)  
   구현 코드는 전개된 식을 바탕으로 합니다.   
     **<center> equation (2)> </center>**
   $$\sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j = \frac{1}{2}\sum_{f=1}^{k}((\sum_{i=1}^{n}v_{i,f}x_i)^2-\sum_{i=1}^{n}v_{i,f}^2x_i^2)$$   
           
   **2. Deep component**  
       Deep component는 MLP Layers로 구성되어 있습니다.   
       구현 코드는 Input dimension이 30-20-10인 3 layer MLP 구조입니다.
  
   

# DeepFM

In [23]:
class DeepFM(nn.Module):
    def __init__(self, input_dims, embedding_dim, mlp_dims, drop_rate=0.1):
        super(DeepFM, self).__init__()
        total_input_dim = int(sum(input_dims)) # n_user + n_movie + n_genre

        # Fm component의 constant bias term과 1차 bias term
        self.bias = nn.Parameter(torch.zeros((1,)))
        self.fc = nn.Embedding(total_input_dim, 1)
        
        self.embedding = nn.Embedding(total_input_dim, embedding_dim)
        self.embedding_dim = len(input_dims) * embedding_dim

        mlp_layers = []
        for i, dim in enumerate(mlp_dims):
            if i==0:
                mlp_layers.append(nn.Linear(self.embedding_dim, dim))
            else:
                mlp_layers.append(nn.Linear(mlp_dims[i-1], dim)) #TODO 1 : linear layer를 넣어주세요.
            mlp_layers.append(nn.ReLU(True))
            mlp_layers.append(nn.Dropout(drop_rate))
        mlp_layers.append(nn.Linear(mlp_dims[-1], 1))
        self.mlp_layers = nn.Sequential(*mlp_layers)

    def fm(self, x):
        # x : (batch_size, total_num_input)
        embed_x = self.embedding(x)

        fm_y = self.bias + torch.sum(self.fc(x), dim=1)
        square_of_sum = torch.sum(embed_x, dim=1) ** 2         #TODO 2 : torch.sum을 이용하여 square_of_sum을 작성해주세요(hint : equation (2))
        sum_of_square = torch.sum(embed_x ** 2, dim=1)         #TODO 3 : torch.sum을 이용하여 sum_of_square을 작성해주세요(hint : equation (2))
        fm_y += 0.5 * torch.sum(square_of_sum - sum_of_square, dim=1, keepdim=True)
        return fm_y
    
    def mlp(self, x):
        embed_x = self.embedding(x)
        
        inputs = embed_x.view(-1, self.embedding_dim)
        mlp_y = self.mlp_layers(inputs)
        return mlp_y

    def forward(self, x):
        #fm component
        fm_y = self.fm(x).squeeze(1)
        
        #deep component
        mlp_y = self.mlp(x).squeeze(1)
        
        y = torch.sigmoid(fm_y + mlp_y)
        return y


# Training

In [24]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [25]:
len(train_dataset)

1361742

In [26]:
device = torch.device('cuda')
# input_dims = [n_user, n_item, n_genre, n_year, n_director, n_writers]
input_dims = [n_user, n_item, n_genre, n_year]
embedding_dim = 10
model = DeepFM(input_dims, embedding_dim, mlp_dims=[30, 20, 10]).to(device)
bce_loss = nn.MSELoss() # Binary Cross Entropy loss
lr, num_epochs = 0.01, 2
optimizer = optim.Adam(model.parameters(), lr=lr)

for e in range(num_epochs) :
    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        model.train()
        optimizer.zero_grad()
        output = model(x)
        loss = bce_loss(output, y.float())
        loss.backward()
        optimizer.step()

 14%|█▎        | 181/1330 [02:12<14:04,  1.36it/s]


KeyboardInterrupt: 

# Evaluation
평가는 모델이 postive instance에 대해 0.5이상, negative instance에 대해 0.5미만의 값을 예측한 Accuracy를 측정하여 진행됩니다.

In [14]:
correct_result_sum = 0
for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    model.eval()
    output = model(x)
    result = torch.round(output)
    correct_result_sum += (result == y).sum().float()

acc = correct_result_sum/len(test_dataset)*100
print("Final Acc : {:.2f}%".format(acc.item()))

Final Acc : 81.63%


# Inference

In [15]:
joined_rating_df = raw_rating_df
joined_rating_df = pd.merge(joined_rating_df, raw_genre_df, left_on='item', right_on='item', how='inner')
joined_rating_df = pd.merge(joined_rating_df, raw_year_df, left_on='item', right_on='item', how='inner')
joined_rating_df = pd.merge(joined_rating_df, raw_director_df, left_on='item', right_on='item', how='inner')
joined_rating_df = pd.merge(joined_rating_df, raw_writer_df, left_on='item', right_on='item', how='inner')


# 5. user, item을 zero-based index로 mapping
users = list(set(joined_rating_df.loc[:,'user']))
users.sort()
items =  list(set((joined_rating_df.loc[:, 'item'])))
items.sort()
genres =  list(set((joined_rating_df.loc[:, 'genre'])))
genres.sort()
years =  list(set((joined_rating_df.loc[:, 'year'])))
years.sort()
directors =  list(set((joined_rating_df.loc[:, 'director'])))
directors.sort()
writers =  list(set((joined_rating_df.loc[:, 'writer'])))
writers.sort()

ans_df = pd.DataFrame(0, index = users, columns= items)
ans_df

if len(users)-1 != max(users):
    users_dict = {users[i]: i for i in range(len(users))}
    joined_rating_df['user']  = joined_rating_df['user'].map(lambda x : users_dict[x])
    users = list(set(joined_rating_df.loc[:,'user']))
    
if len(items)-1 != max(items):
    items_dict = {items[i]: i for i in range(len(items))}
    joined_rating_df['item']  = joined_rating_df['item'].map(lambda x : items_dict[x])
    items =  list(set((joined_rating_df.loc[:, 'item'])))

if len(genres)-1 != max(genres):
    genres_dict = {genres[i]: i for i in range(len(genres))}
    joined_rating_df['genre']  = joined_rating_df['genre'].map(lambda x : genres_dict[x])
    genres =  list(set((joined_rating_df.loc[:, 'genre'])))

if len(years)-1 != max(years):
    years_dict = {years[i]: i for i in range(len(years))}
    joined_rating_df['year']  = joined_rating_df['year'].map(lambda x : years_dict[x])
    years =  list(set((joined_rating_df.loc[:, 'year'])))

if len(directors)-1 != max(directors):
    directors_dict = {directors[i]: i for i in range(len(directors))}
    joined_rating_df['director']  = joined_rating_df['director'].map(lambda x : directors_dict[x])
    directors =  list(set((joined_rating_df.loc[:, 'director'])))
    
if len(writers)-1 != max(writers):
    writers_dict = {writers[i]: i for i in range(len(writers))}
    joined_rating_df['writer']  = joined_rating_df['writer'].map(lambda x : writers_dict[x])
    writers =  list(set((joined_rating_df.loc[:, 'writer'])))
    
joined_rating_df = joined_rating_df.sort_values(by=['user'])
joined_rating_df.reset_index(drop=True, inplace=True)

data = joined_rating_df



n_data = len(data)
n_user = len(users)
n_item = len(items)
n_genre = len(genres)
n_year = len(years)
n_director = len(directors)
n_writers = len(writers)

#6. feature matrix X, label tensor y 생성
user_col = torch.tensor(data.loc[:,'user'])
item_col = torch.tensor(data.loc[:,'item'])
genre_col = torch.tensor(data.loc[:,'genre'])
year_col = torch.tensor(data.loc[:,'year'])
director_col = torch.tensor(data.loc[:,'director'])
writer_col = torch.tensor(data.loc[:,'writer'])

offsets = [0, n_user, n_user+n_item, n_user+n_item+n_genre, n_user+n_item+n_genre+n_year, n_user+n_item+n_genre+n_year+n_director]
for col, offset in zip([user_col, item_col, genre_col, year_col, director_col, writer_col], offsets):
    col += offset

# X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1), year_col.unsqueeze(1), director_col.unsqueeze(1), writer_col.unsqueeze(1)], dim=1)
X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1), year_col.unsqueeze(1)], dim=1)
y = torch.tensor(list(data.loc[:,'rating']))

submission_dataset = RatingDataset(X, y)
submission_dataloader = DataLoader(submission_dataset, batch_size=512, shuffle=False)


In [16]:
first = True
for x, _ in tqdm(submission_dataloader):
    users = x[:,0]
    items = x[:,1]
    x = x.to(device)
    model.eval()
    output = model(x).cpu().detach().numpy()
    tmp = pd.DataFrame({"user":users, "item": items-n_user, "score":output})
    if first:
        first = False
        user_items = tmp
    else:
        user_items = pd.concat([user_items, tmp], axis = 0, sort=False)

100%|██████████| 2956/2956 [03:28<00:00, 14.20it/s]


In [17]:
tmp = user_items.groupby("user")["item"]

In [18]:
reversed_users_dict = {}
reversed_items_dict = {}
for k,v in users_dict.items():
    reversed_users_dict[v]=k
for k,v in items_dict.items():
    reversed_items_dict[v]=k


In [19]:
ans_user = []; ans_item = []
for u, grouped in tmp:
    for i in range(10):
        ans_user.append(reversed_users_dict[u]) 
        ans_item.append(reversed_items_dict[grouped.iloc[i]])

In [20]:
pd.DataFrame({"user":ans_user,"item":ans_item}).to_csv(
    "output/submission.csv", index=False
)

In [154]:
# first = True
# for x, _ in tqdm(submission_dataloader):
#     ans_users = x[:,0]
#     ans_items = x[:,1]
#     x = x.to(device)
#     model.eval()
#     output = model(x).cpu().detach().numpy()
#     ans_df.iloc[ans_users.tolist(),(ans_items-n_user).tolist()] = output.tolist()

100%|██████████| 2956/2956 [09:47<00:00,  5.04it/s]


In [160]:
# ans_df

,1,2,6,7,10,11,16,17,19,21,...,109374,109487,110102,111362,111759,112175,112183,112290,112623,112852
11,0.004332,0.254788,0.0,0.0,0.000000,0.0,1.000000,1.0,0.000000e+00,0.0,...,0.0,0.0,0.999999,0.105972,0.0,0.0,0.0,0.0,0.0,0.0
14,0.004332,0.254788,0.0,0.0,0.000000,0.0,1.000000,1.0,0.000000e+00,0.0,...,0.0,0.0,0.999999,0.105972,0.0,0.0,0.0,0.0,0.0,0.0
18,0.004332,0.254788,0.0,0.0,0.000000,0.0,1.000000,1.0,0.000000e+00,0.0,...,0.0,0.0,0.999999,0.105972,0.0,0.0,0.0,0.0,0.0,0.0
25,0.004332,0.254788,0.0,0.0,0.000000,0.0,1.000000,1.0,0.000000e+00,0.0,...,0.0,0.0,0.999999,0.105972,0.0,0.0,0.0,0.0,0.0,0.0
31,0.004332,0.254788,0.0,0.0,0.000000,0.0,1.000000,1.0,0.000000e+00,0.0,...,0.0,0.0,0.999999,0.105972,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138473,0.899496,0.996541,0.0,0.0,0.006125,0.0,0.990171,0.0,4.492531e-12,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
138475,0.899496,0.996541,0.0,0.0,0.006125,0.0,0.990171,0.0,4.492531e-12,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
138486,0.899496,0.996541,0.0,0.0,0.006125,0.0,0.990171,0.0,4.492531e-12,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
138492,0.899496,0.996541,0.0,0.0,0.006125,0.0,0.990171,0.0,4.492531e-12,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# from utils import generate_submission_file

# dataset = RatingDataset(X, y)
# sbm_loader =  DataLoader(dataset, batch_size=1024, shuffle=True)

# tmp = X[:,0]
# ans = torch.Tensor().to(device)
# for x, _ in sbm_loader:
#     x = x.to(device)
#     model.eval()
#     output = model(x).to(device)
#     for idx in range(len(x)):
#         if int(x[idx][1]) in user_items.get(int(x[idx][0]),set()):
#             output[idx] = 0
#     ans = torch.cat([ans,output], dim=0)

# # generate_submission_file("/opt/ml/input/code/output/train_ratings.csv",pred_list)

In [26]:
# a = {x:[] for x in range(31360)}
# for i in range(len(X)):
#     a[int(tmp[i])].append((X[i][1],ans[i]))

In [27]:
# for i in a.keys():
#     a[i].sort(key = lambda x: -x[1])
#     a[i] = a[i][:10]

In [29]:
# generate_submission_file("/opt/ml/input/code/data/train/train_ratings.csv",preds)